[View in Colaboratory](https://colab.research.google.com/github/shashank2806/mnist-digits-classifiers/blob/master/convolutional_neural_network.ipynb)

#Convolutional Neural Network

We earlier used linear model on mnist data and got around 92% test accuracy.

In this notebook we will implement CNN in TensorFlow for classification.

##Imports

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import math

This notebook was developed using Google Colab using python 3.6 and TensorFlow version:

In [3]:
tf.__version__

'1.10.0'

###Load Data